In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


# PARAMETERS
train_dir = r"E:\LY Project\Multi Cancer\Data"  # Folder with 2 subfolders (Benign and Malignant)
img_height, img_width = 224, 224    # DenseNet201 input size
batch_size = 32
num_classes = 2  # Binary classification
epochs = 20


# -------------------------------
# LOAD DATA - NO AUGMENTATION, ONLY RESIZE AND RESCALE
# -------------------------------
print("Loading training data without augmentation (only rescaling)...")


datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% validation split
)


train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)


validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


# -------------------------------
# LOAD DENSENET201 BASE MODEL AND FREEZE LAYERS
# -------------------------------
print("Loading DenseNet201 pre-trained base model...")
base_model = DenseNet201(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_height, img_width, 3)
)


# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False
print(f"Frozen {len(base_model.layers)} layers of DenseNet201 base.")


# -------------------------------
# ADD CLASSIFIER ON TOP
# -------------------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)  # DenseNet201 uses GAP instead of Flatten
x = Dense(512, activation='relu')(x)  # Larger dense layer for complex features
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


# Display model summary
print("\nModel Architecture Summary:")
model.summary()


# -------------------------------
# COMPILE MODEL
# -------------------------------
print("\nCompiling model...")
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# -------------------------------
# CALLBACKS
# -------------------------------
checkpoint_filepath = 'best_densenet201_cancer_model.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, 
    monitor='val_accuracy', 
    save_best_only=True, 
    verbose=1
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)


# -------------------------------
# TRAIN MODEL
# -------------------------------
print(f"\nTraining for {epochs} epochs...")
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


print("\nTraining completed.")
print(f"Best model saved at {checkpoint_filepath}")


# -------------------------------
# PLOT TRAINING HISTORY
# -------------------------------
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()


# -------------------------------
# LOAD BEST MODEL FROM CHECKPOINT
# -------------------------------
print("\nLoading best model for evaluation...")
model.load_weights(checkpoint_filepath)


# -------------------------------
# EVALUATE ON VALIDATION SET
# -------------------------------
print("\nEvaluating model on validation set...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


# -------------------------------
# PREDICTIONS AND CONFUSION MATRIX
# -------------------------------
print("\nPredicting classes on validation data...")
validation_generator.reset()  # Reset generator for consistent predictions
val_steps = validation_generator.samples // validation_generator.batch_size + 1


# Predict probabilities for validation data
y_pred_probs = model.predict(validation_generator, steps=val_steps)
y_pred = np.argmax(y_pred_probs, axis=1)


# True labels
y_true = validation_generator.classes[:len(y_pred)]  # Match predictions length


# Labels for confusion matrix and classification report
class_labels = list(validation_generator.class_indices.keys())


# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)


# Display confusion matrix as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues', 
    xticklabels=class_labels, 
    yticklabels=class_labels,
    cbar_kws={'label': 'Count'}
)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix - DenseNet201', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


# Print classification report
print("\n" + "="*60)
print("CLASSIFICATION REPORT - DENSENET201")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_labels))


# -------------------------------
# ADDITIONAL METRICS
# -------------------------------
from sklearn.metrics import roc_curve, auc, roc_auc_score

# For binary classification, extract probabilities for positive class
y_pred_probs_positive = y_pred_probs[:len(y_true), 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_probs_positive)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nROC-AUC Score: {roc_auc:.4f}")
